In [1]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error

import warnings
warnings.filterwarnings("ignore")

In [2]:
try:
    connection = mysql.connector.connect(
        host='lucida-pricevolume.rwlb.rds.aliyuncs.com',
        database='lucida_data_center',
        user='back_test_read',
        password='lucida_read@123'
    )
    if connection.is_connected():
        db_info = connection.get_server_info()
        print("成功连接到 MySQL 数据库，版本：", db_info)
except Error as e:
    print("连接错误：", e)
    

成功连接到 MySQL 数据库，版本： 8.0.13


In [3]:
cursor = connection.cursor()

In [4]:
table_columns_q = "SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = %s AND TABLE_SCHEMA = %s ORDER BY ORDINAL_POSITION;"
cursor.execute(table_columns_q, ("binance_spot_kline_daily_2023","lucida_data_center"))
table_columns = cursor.fetchall()
table_columns = [x[0] for x in table_columns]
table_columns

['current_time',
 'symbol',
 'open_time',
 'open_time_date',
 'open',
 'high',
 'low',
 'close',
 'volume',
 'close_time',
 'close_time_date',
 'amount',
 'number_of_trades',
 'buy_volume',
 'buy_amount']

In [5]:
daily_year = 2022
end_year = 2025
all_data_tmp = pd.DataFrame()
while daily_year <= end_year:
    table_name = f"binance_spot_kline_daily_{str(daily_year)}"
    daily_year += 1
    query = f'select * from {table_name};'
    cursor.execute(query)
    records = cursor.fetchall()
    data = pd.DataFrame(records)
    all_data_tmp = pd.concat([all_data_tmp,data],axis=0)
all_data_tmp.columns = table_columns

In [6]:
all_data_tmp = all_data_tmp[["close_time_date","symbol","close"]]
all_data_tmp["close"] = all_data_tmp["close"].astype(float)

In [7]:
all_data_tmp

,close_time_date,symbol,close
0,2022-01-02 00:00:00,BNBETH,0.14000
1,2022-01-03 00:00:00,BNBETH,0.13870
2,2022-01-04 00:00:00,BNBETH,0.13590
3,2022-01-05 00:00:00,BNBETH,0.13390
4,2022-01-06 00:00:00,BNBETH,0.13400
...,...,...,...
45834,2025-02-05 08:00:00,PNUTBRL,1.13400
45835,2025-02-05 08:00:00,ACXUSDC,0.34790
45836,2025-02-05 08:00:00,RAYUSDC,6.52400
45837,2025-02-05 08:00:00,HBARFDUSD,0.26039


In [8]:
# 仅保留以quote=USDT的symbol
all_data_tmp = all_data_tmp[all_data_tmp['symbol'].str.endswith('USDT')]
all_data_tmp.reset_index(inplace=True,drop=True)

In [9]:
all_data_tmp

,close_time_date,symbol,close
0,2022-01-02 00:00:00,ETHUSDT,3765.54000
1,2022-01-03 00:00:00,ETHUSDT,3828.27000
2,2022-01-04 00:00:00,ETHUSDT,3765.89000
3,2022-01-05 00:00:00,ETHUSDT,3785.11000
4,2022-01-06 00:00:00,ETHUSDT,3540.63000
...,...,...,...
370089,2025-02-05 08:00:00,VELODROMEUSDT,0.07700
370090,2025-02-05 08:00:00,THEUSDT,0.52040
370091,2025-02-05 08:00:00,1000CATUSDT,0.01111
370092,2025-02-05 08:00:00,USUALUSDT,0.28120


In [10]:
# 1. 判断是否进入牛市周期 BTCUSDT PRMA120=Pclose/MA120−1 > 0.02
bullish_cycle_data = all_data_tmp[all_data_tmp["symbol"]=="BTCUSDT"][["close_time_date","close"]]
bullish_cycle_data.sort_values(by="close_time_date",inplace=True)
bullish_cycle_data.reset_index(inplace=True,drop=True)
bullish_cycle_data["MA120"] = bullish_cycle_data["close"].rolling(window=120).mean()
bullish_cycle_data.dropna(inplace=True)
bullish_cycle_data.reset_index(inplace=True,drop=True)
bullish_cycle_data["PRMA120"] = (bullish_cycle_data["close"] / bullish_cycle_data["MA120"]) -1 

In [11]:
# 2. 计算指标
new_all_data = pd.DataFrame()
symbols = list(np.unique(all_data_tmp["symbol"]))
for s in symbols:
    df = all_data_tmp[all_data_tmp["symbol"]==s]
    df.sort_values(by="close_time_date",inplace=True)
    df.reset_index(inplace=True,drop=True)
    df["MA7"] = df["close"].rolling(window=7).mean()
    # df["MA30"] = df["close"].rolling(window=30).mean()
    df.dropna(inplace=True)
    df.reset_index(inplace=True,drop=True)
    df["PRMA7"] = (df["close"] / df["MA7"]) -1
    # df["PRMA30"] = (df["close"] / df["MA30"]) -1
    new_all_data = pd.concat([new_all_data,df], axis=0)


In [12]:
new_all_data.sort_values(by="close_time_date")

,close_time_date,symbol,close,MA7,PRMA7
0,2022-01-08 00:00:00,ATMUSDT,6.640000,7.007143,-0.052396
0,2022-01-08 00:00:00,OOKIUSDT,0.026340,0.029860,-0.117883
0,2022-01-08 00:00:00,DEXEUSDT,9.260000,10.980000,-0.156648
0,2022-01-08 00:00:00,ARDRUSDT,0.212000,0.240200,-0.117402
0,2022-01-08 00:00:00,ORNUSDT,5.013000,5.695286,-0.119798
...,...,...,...,...,...
1124,2025-02-05 08:00:00,HIVEUSDT,0.327800,0.372657,-0.120371
1124,2025-02-05 08:00:00,HIGHUSDT,0.894000,1.085429,-0.176362
1124,2025-02-05 08:00:00,ACMUSDT,1.060000,1.158714,-0.085193
1124,2025-02-05 08:00:00,IOSTUSDT,0.005021,0.005504,-0.087754


# backdrop1(2023.1 - 2023.6)

In [13]:
backdrop1_bullish_cycle_data = bullish_cycle_data[
    (bullish_cycle_data["close_time_date"] >= "2023-01-01") & 
    (bullish_cycle_data["close_time_date"] <= "2023-06-30")
]

In [14]:
backdrop1_bullish_cycle_data[backdrop1_bullish_cycle_data["PRMA120"]>0.02]

,close_time_date,close,MA120,PRMA120
257,2023-01-13,18846.62,18145.417750,0.038643
258,2023-01-14,19930.01,18147.318833,0.098234
259,2023-01-15,20954.92,18156.915667,0.154101
260,2023-01-16,20871.50,18163.231333,0.149107
261,2023-01-17,21185.65,18177.976917,0.165457
...,...,...,...,...
421,2023-06-26,30462.66,27045.324333,0.126356
422,2023-06-27,30267.99,27101.267167,0.116848
423,2023-06-28,30692.44,27161.270083,0.130008
424,2023-06-29,30077.41,27219.068750,0.105012


In [15]:
# 3. 筛选 PRMA7>0.05
backdrop1_data = new_all_data[
    (new_all_data["close_time_date"] >= "2023-01-01") & 
    (new_all_data["close_time_date"] <= "2023-06-30")
]
backdrop1_data = backdrop1_data[(backdrop1_data["PRMA7"]>0.05)]
backdrop1_data.sort_values(by="close_time_date",inplace=True)
backdrop1_data.reset_index(inplace=True,drop=True)
backdrop1_data

,close_time_date,symbol,close,MA7,PRMA7
0,2023-01-01,TWTUSDT,1.421900,1.348429,0.054487
1,2023-01-01,DENTUSDT,0.000684,0.000648,0.056254
2,2023-01-02,JASMYUSDT,0.003301,0.002997,0.101435
3,2023-01-02,LDOUSDT,1.080000,0.994714,0.085739
4,2023-01-02,HOOKUSDT,1.336600,1.218586,0.096845
...,...,...,...,...,...
8707,2023-06-30,MDTUSDT,0.048000,0.043379,0.106537
8708,2023-06-30,XLMUSDT,0.102100,0.095557,0.068471
8709,2023-06-30,VGXUSDT,0.159800,0.132271,0.208122
8710,2023-06-30,XVGUSDT,0.003120,0.002123,0.469421


In [16]:
backdrop1_finall_data = pd.DataFrame()
for date in list(np.unique(backdrop1_data["close_time_date"])):
    df = backdrop1_data[backdrop1_data["close_time_date"]==date]
    df.sort_values(by="PRMA7",ascending=False)
    df.reset_index(inplace=True,drop=True)
    backdrop1_finall_data=pd.concat([backdrop1_finall_data,df.iloc[:5,:]],axis=0)

In [53]:
backdrop1_finall_data.to_csv("backdrop1(2023.1 - 2023.6).csv",index=False)

# backdrop2(2023.10 - 2024.4)

In [17]:
backdrop2_bullish_cycle_data = bullish_cycle_data[
    (bullish_cycle_data["close_time_date"] >= "2023-10-01 00:00:00") & 
    (bullish_cycle_data["close_time_date"] <= "2024-04-30 00:00:00")
]

In [18]:
backdrop2_bullish_cycle_data[backdrop2_bullish_cycle_data["PRMA120"]>0.02]

,close_time_date,close,MA120,PRMA120
538,2023-10-21,29669.04,28195.318583,0.052268
539,2023-10-22,29909.80,28188.829417,0.061052
540,2023-10-23,29992.46,28184.371333,0.064152
541,2023-10-24,33069.99,28206.099083,0.172441
542,2023-10-25,33922.73,28236.555250,0.201376
...,...,...,...,...
726,2024-04-26,64498.34,55872.584583,0.154383
727,2024-04-27,63770.01,56049.303333,0.137748
728,2024-04-28,63461.98,56227.595250,0.128663
729,2024-04-29,63118.62,56402.414750,0.119077


In [19]:
# 3. 筛选 PRMA7>0.05
backdrop2_data = new_all_data[
    (new_all_data["close_time_date"] >= "2023-10-01") & 
    (new_all_data["close_time_date"] <= "2024-04-30")
]
backdrop2_data = backdrop2_data[(backdrop2_data["PRMA7"]>0.05)]
backdrop2_data.sort_values(by="close_time_date",inplace=True)
backdrop2_data.reset_index(inplace=True,drop=True)
backdrop2_finall_data = pd.DataFrame()
for date in list(np.unique(backdrop2_data["close_time_date"])):
    df = backdrop2_data[backdrop2_data["close_time_date"]==date]
    df.sort_values(by="PRMA7",ascending=False)
    df.reset_index(inplace=True,drop=True)
    backdrop2_finall_data=pd.concat([backdrop2_finall_data,df.iloc[:5,:]],axis=0)
# backdrop2_finall_data.to_csv("backdrop2(2023.10 - 2024.4).csv",index=False)
backdrop2_finall_data.to_csv("backdrop2(2023.10 - 2024.4)_副本.csv",index=False)

# backdrop3(2024.9 - 2024.12 )

In [20]:
# 3. 筛选 PRMA7>0.05
backdrop3_data = new_all_data[
    (new_all_data["close_time_date"] >= "2024-09-01") & 
    (new_all_data["close_time_date"] <= "2024-12-31")
]
backdrop3_data = backdrop3_data[(backdrop3_data["PRMA7"]>0.05)]
backdrop3_data.sort_values(by="close_time_date",inplace=True)
backdrop3_data.reset_index(inplace=True,drop=True)
backdrop3_finall_data = pd.DataFrame()
for date in list(np.unique(backdrop3_data["close_time_date"])):
    df = backdrop3_data[backdrop3_data["close_time_date"]==date]
    df.sort_values(by="PRMA7",ascending=False)
    df.reset_index(inplace=True,drop=True)
    backdrop3_finall_data=pd.concat([backdrop3_finall_data,df.iloc[:5,:]],axis=0)
# backdrop3_finall_data.to_csv("backdrop3(2024.9 - 2024.12).csv",index=False)
backdrop3_finall_data.to_csv("backdrop3(2024.9 - 2024.12)_副本.csv",index=False)

In [21]:
backdrop3_data.sort_values(by="close_time_date")

,close_time_date,symbol,close,MA7,PRMA7
0,2024-09-01 23:59:59,LOKAUSDT,0.214000,0.200629,0.066648
1,2024-09-01 23:59:59,STMXUSDT,0.005936,0.005651,0.050487
2,2024-09-01 23:59:59,HARDUSDT,0.146300,0.122586,0.193451
14,2024-09-02 23:59:59,VIDTUSDT,0.049380,0.046813,0.054838
13,2024-09-02 23:59:59,BELUSDT,0.515700,0.489471,0.053586
...,...,...,...,...,...
11210,2024-12-30 23:59:59,BSWUSDT,0.076300,0.071743,0.063521
11209,2024-12-30 23:59:59,FIROUSDT,2.321000,2.181429,0.063982
11223,2024-12-30 23:59:59,ACAUSDT,0.110700,0.102114,0.084079
11215,2024-12-30 23:59:59,MBLUSDT,0.003567,0.003321,0.074028


In [22]:
all_data_tmp[
    (all_data_tmp["close_time_date"] >= "2023-10-01") & 
    (all_data_tmp["close_time_date"] <= "2024-04-30")
]

,close_time_date,symbol,close
102543,2023-10-01,ETHUSDT,1670.89
102544,2023-10-02,ETHUSDT,1733.79
102545,2023-10-03,ETHUSDT,1662.40
102546,2023-10-04,ETHUSDT,1656.88
102547,2023-10-05,ETHUSDT,1646.58
...,...,...,...
291959,2024-04-26,OMNIUSDT,21.78
291960,2024-04-27,OMNIUSDT,20.61
291961,2024-04-28,OMNIUSDT,20.42
291962,2024-04-29,OMNIUSDT,20.64


# 同时存在3个backdrop的symbol

In [24]:
res = pd.merge(backdrop1_finall_data,backdrop2_finall_data,on="symbol",how="inner")
res = pd.merge(res,backdrop3_finall_data,on="symbol",how="inner")
res

,close_time_date_x,symbol,close_x,MA7_x,PRMA7_x,close_time_date_y,close_y,MA7_y,PRMA7_y,close_time_date,close,MA7,PRMA7
0,2023-01-01,TWTUSDT,1.4219,1.348429,0.054487,2023-10-06,0.9750,0.826400,0.179816,2024-09-25 23:59:59,0.8864,0.840457,0.054664
1,2023-01-01,TWTUSDT,1.4219,1.348429,0.054487,2023-10-06,0.9750,0.826400,0.179816,2024-10-04 23:59:59,1.0743,1.020729,0.052484
2,2023-01-01,TWTUSDT,1.4219,1.348429,0.054487,2023-10-06,0.9750,0.826400,0.179816,2024-10-06 23:59:59,1.0887,1.035429,0.051449
3,2023-01-01,TWTUSDT,1.4219,1.348429,0.054487,2023-10-08,1.0137,0.882086,0.149208,2024-09-25 23:59:59,0.8864,0.840457,0.054664
4,2023-01-01,TWTUSDT,1.4219,1.348429,0.054487,2023-10-08,1.0137,0.882086,0.149208,2024-10-04 23:59:59,1.0743,1.020729,0.052484
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3884,2023-06-30,RAYUSDT,0.2181,0.199471,0.093390,2024-03-19,2.6496,1.935157,0.369191,2024-10-28 23:59:59,3.2640,2.969286,0.099254
3885,2023-06-30,RAYUSDT,0.2181,0.199471,0.093390,2024-03-19,2.6496,1.935157,0.369191,2024-11-19 23:59:59,5.5210,5.177286,0.066389
3886,2023-06-30,RAYUSDT,0.2181,0.199471,0.093390,2024-03-19,2.6496,1.935157,0.369191,2024-11-21 23:59:59,6.3030,5.529143,0.139960
3887,2023-06-30,RAYUSDT,0.2181,0.199471,0.093390,2024-03-19,2.6496,1.935157,0.369191,2024-11-22 23:59:59,6.0660,5.706000,0.063091


In [27]:
symbols = np.unique(res["symbol"])

In [31]:
symbols

array(['AAVEUSDT', 'ADAUSDT', 'AGLDUSDT', 'ALCXUSDT', 'AMBUSDT',
       'APEUSDT', 'APTUSDT', 'ARDRUSDT', 'ATMUSDT', 'AVAXUSDT', 'AXSUSDT',
       'BETAUSDT', 'BICOUSDT', 'BNXUSDT', 'C98USDT', 'CELOUSDT',
       'CELRUSDT', 'CFXUSDT', 'CHESSUSDT', 'CKBUSDT', 'CLVUSDT',
       'COSUSDT', 'CTKUSDT', 'CTXCUSDT', 'DARUSDT', 'DATAUSDT',
       'DEGOUSDT', 'DEXEUSDT', 'DFUSDT', 'DIAUSDT', 'DOGEUSDT',
       'DUSKUSDT', 'DYDXUSDT', 'EDUUSDT', 'EGLDUSDT', 'ELFUSDT',
       'ENJUSDT', 'FIDAUSDT', 'FIOUSDT', 'FIROUSDT', 'FISUSDT', 'FLMUSDT',
       'FLOKIUSDT', 'FLOWUSDT', 'FLUXUSDT', 'FORTHUSDT', 'FTMUSDT',
       'FUNUSDT', 'FXSUSDT', 'GALAUSDT', 'GLMUSDT', 'GMTUSDT', 'GNOUSDT',
       'GRTUSDT', 'GTCUSDT', 'HARDUSDT', 'HBARUSDT', 'HFTUSDT',
       'HIVEUSDT', 'HOTUSDT', 'ICPUSDT', 'ICXUSDT', 'IMXUSDT', 'IOTAUSDT',
       'IOTXUSDT', 'JASMYUSDT', 'JSTUSDT', 'JUVUSDT', 'KAVAUSDT',
       'KEYUSDT', 'LDOUSDT', 'LINAUSDT', 'LITUSDT', 'LOKAUSDT', 'LPTUSDT',
       'LQTYUSDT', 'LRCUSDT', 'LSKUSDT',